# Import the data 

In [5]:
import pandas as pd 
import numpy as np
import re
import time
from tqdm import tqdm

In [6]:
brands = pd.read_csv('behold_brands USC.csv')
product = pd.read_excel('Behold+product+data+04262021.xlsx')
addtional_tags = pd.read_csv('usc_additional_tags USC.csv')

In [7]:
brands.head()

,brand_id,brand,brand_value,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio
0,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Handmade / Artisan Crafted,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
1,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Sustainable,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
2,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Women Owned,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
3,01ESVD80YYAJQEQPADZ7HTHZXA,Warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Behold Warm! Founded by specialty store owner,..."
4,01EFJFZ329YVC8SK87R2BNJGV0,Vaara,Emerging,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
brands.describe().T

,count,unique,top,freq
brand_id,162,74,01EFJFZ29KVBK14BDJNFBDK1G7,5
brand,162,74,Zonarch,5
brand_value,154,6,Women Owned,47
bio,157,71,lemlem is a women's resort wear brand made ent...,5
quote,157,71,This sustainable California swimwear brand off...,5
quote_attribute,154,59,Vogue,13
intro,154,70,"Founded in 2007 by Liya Kebede, lemlem is enti...",5
lifestyle_copy,156,66,...,8
short_bio,157,71,Á La Holiday,5
listing_bio,157,71,Behold Studio 189! Founded by Abrima Erwiyah a...,5


In [9]:
brands.listing_bio[1]

"Behold Mari Giudicelli! This Brazilian shoe designer is a maven of style, and her designs are packed with easy-going elegance. You'll want to pair these covet worthy shoes with every outfit."

In [10]:
brands.isna().sum()

brand_id           0
brand              0
brand_value        8
bio                5
quote              5
quote_attribute    8
intro              8
lifestyle_copy     6
short_bio          5
listing_bio        5
dtype: int64

In [11]:
print(f' Number of Brands in the Brand file: {len(brands.brand.unique())}')

 Number of Brands in the Brand file: 74


In [12]:
product.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,NaN,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False


In [13]:
product.describe().T

,count,unique,top,freq
product_id,61355,61355,01EMPJ10N8B1Z0SXQP6593W2J5,1
brand,61355,386,7 For All Mankind,9011
brand_category,60896,632,Unknown,53249
name,61354,42615,ANKLE SKINNY,247
details,9200,7034,True to size.,265
created_at,61355,61349,2020-08-28 21:52:04.844 UTC,2
brand_canonical_url,61355,58156,https://www.7forallmankind.com/airweft-denim-s...,13
description,51238,42210,product name by ancient greek sandals,127
brand_description,51234,42550,product name by ancient greek sandals,127
brand_name,61354,42602,ANKLE SKINNY,247


In [14]:
product.isna().sum()

product_id                 0
brand                      0
brand_category           459
name                       1
details                52155
created_at                 0
brand_canonical_url        0
description            10117
brand_description      10121
brand_name                 1
product_active             0
dtype: int64

In [15]:
print(f' Number of Brands in the Product file: {len(product.brand.unique())}')

 Number of Brands in the Product file: 386


In [16]:
print(f'Number of Brand Category: {len(product.brand_category.unique())}')

Number of Brand Category: 633


In [17]:
product.brand_canonical_url[0]

'https://two-nyc.myshopify.com/products/white-khadi-stripe-shirt'

# Cleaning

## Cleaning Tasks
1. fill missing values
2. remove \n
3. remove punctuation
4. change to lower case
5. remove stopwords
6. create named entities


In [18]:
# create a copy of the original dataset
product_copy = product.copy()

In [19]:
# fill missing values
product_copy.fillna('None',inplace = True)

In [20]:
# remove '\n'
product_copy.description = product_copy.description.str.replace('\n',' ')
product_copy.description = product_copy.description.str.replace('\r',' ')

# # remove punctuations
product_copy.description = product_copy.description.str.replace(r'[^\w\s]|_',' ')

#change all the words to lower case 
product_copy.description = product_copy.description.str.lower()

In [21]:
product_copy.description[1]

'mid length dress with ruffles and adjustable straps  bias cut  side seam invisible zipper made in new york model wears size small 100  rose sylk rose sylk is an organic cellulose fiber made from the natural waste of rose bushes and stems '

In [22]:
## remove stopwords
from nltk.corpus import stopwords
from nltk import word_tokenize


custom_sw = ['size', 'height', 'waist','bust' 'hip','measurement','model','wear','small','medium','large']
stopwords_list = stopwords.words('english') + custom_sw
stopwords_list.index('in')
print(len(stopwords_list))
## remove 'in' from stopwords because it is useful to create the 'made_in' feature
stopwords_list.pop(90)
print(len(stopwords_list))

189
188


In [23]:
# define a function to remove stopwords
def remove_sw(text):
    text = str(text)
    # split sentence into words
    words = word_tokenize(text)
    
    new_words = []
    # remove stopwords
    for w in words:
        if w in stopwords_list:
            continue
        new_words.append(w)
    
    return ' '.join(new_words)

In [24]:
%%time
product_copy['description'] = pd.DataFrame(product_copy['description'].apply(remove_sw))

Wall time: 18.1 s


## Create Named Entities

In [25]:
product_copy['description'] = product_copy['description'].str.\
                                replace(r'\bnew\b\s\byork\b\s(?:\bcity\b)?','new_york_city ')



In [26]:
## check the result of replacement
product_copy['description'].apply(lambda x: re.search(r'\bnew\b\s\byork\b\s(?:\bcity\b)?',x)).sum()

0

In [27]:
product_copy['description'][1]

'mid length dress ruffles adjustable straps bias cut side seam invisible zipper made in new_york_city wears 100 rose sylk rose sylk organic cellulose fiber made natural waste rose bushes stems'

In [28]:
product_copy['description'] = product_copy['description'].str.\
                                replace(r'\b(?:the\s)?usa?\b','USA')
product_copy['description'] = product_copy['description'].str.\
                                replace(r'\b(?:the\s)?united\sstates\b','USA')

In [29]:
## check the result 
product_copy['description'].apply(lambda x: re.search(r'\b(?:the\s)?usa?\b',x)).sum()

0

# Categorizing-- Create Features

1. user type, women, men, or children
2. color
3. category - joyi
4. occasions? sports, casual,cozy, formal, swim, holiday, business 
5. origin- USA, China, Europe, Other 
6. style ： boyfriend, destressed, 
7. care: no machine wash,dry wash, hand wash, tumble dry
8. fabric: ploy, chiffon, ...


## user type

In [30]:
## because a large proportion of clothings is for women
## we suppose that if there are no key words indicate it is for children or men, 
## we treat this product as women's clothing
## for the products that doesn't have description
## also treat it as women's clothing 

def isWomensClothing(txt):
    """ Function to determine whether it is an article of women's clothing """

    txt = str(txt)
    val = False
    if re.search(r'\b(Girl?|boy?|men|man|Gir?|baby|kid?)\b', txt, re.IGNORECASE ):
        val = True
    return val


In [31]:
product_copy['is_womens_clothing'] = pd.DataFrame(
                                                [product_copy.description.\
                                                  apply(isWomensClothing),
                                                product_copy.name.apply(isWomensClothing),
                                                product_copy.details.\
                                                 apply(isWomensClothing)]
                                                    ).all()
    
# any return true is false women
# all return false is true women

product_copy['is_womens_clothing'].value_counts()

False    61355
Name: is_womens_clothing, dtype: int64

In [32]:
def isChildrenClothing(txt):
    """ Function to determine whether it is an article of women's clothing """

    txt = str(txt)
    val = False
    if re.search(r'\b(Girl?|boy?|Gir?|baby|kid?)\b', txt, re.IGNORECASE ):
        val = True
    return val

In [33]:
product_copy['is_children_clothing'] = np.nan
product_copy['is_children_clothing'] = pd.DataFrame(
                                                  [product_copy.description.\
                                                  apply(isChildrenClothing),
                                                product_copy.name.apply(isChildrenClothing),
                                                product_copy.details.\
                                                 apply(isChildrenClothing)]
                                                    ).any()

product_copy['is_children_clothing'].value_counts()

False    60294
True      1061
Name: is_children_clothing, dtype: int64

In [34]:
def isMenClothing(txt):
    """ Function to determine whether it is an article of women's clothing """

    txt = str(txt)
    val = False
    if re.search(r'\b(man|men|man\'s|men\'s)\b', txt, re.IGNORECASE ):
        val = True
    return val

In [35]:
product_copy['is_men_clothing'] = pd.DataFrame(  [product_copy.description.\
                                                  apply(isMenClothing),
                                                product_copy.name.apply(isMenClothing),
                                                product_copy.details.\
                                                 apply(isMenClothing)]
                                                    ).any()
product_copy['is_men_clothing'].value_counts()

False    60501
True       854
Name: is_men_clothing, dtype: int64

In [36]:
# def user_type(txt):
#     txt = str(txt)
#     user_type = 'women'
#     if re.search(r'\b(man|men|man\'s|men\'s)\b', txt, re.IGNORECASE ):
#         user_type = 'men'
#     if re.search(r'\b(Girl?|boy?|Gir?|baby|kid?)\b', txt, re.IGNORECASE ):
#         user_type = 'children'
#     return user_type

    

In [37]:

conditions = [(product_copy.is_womens_clothing == True),
              (product_copy.is_men_clothing == True),
              (product_copy.is_children_clothing == True)]
              
values = ['women','men','children']

product_copy['user_type']= np.select(conditions, values)

product_copy['user_type'].value_counts()

0           59450
children     1051
men           854
Name: user_type, dtype: int64

## category

In [38]:

# def findCategory(txt):
#     """ Function to determine the article of clothing """
    
#     txt = str(txt)
#     val = np.nan
#     if re.search(r'pants|trousers|jeans|shorts|leggings|skirt|jumpsuit', txt, re.IGNORECASE ):
#         val = "Bottom"
#     elif re.search(r'\bdress\b|gown|jumpsuit', txt, re.IGNORECASE ):
#         val = "One Piece"
#     elif re.search(r'shoes|sneakers|heels|pumps', txt, re.IGNORECASE ):
#         val = "Shoe"
#     elif re.search(r'purse|handbag|tote|clutch', txt, re.IGNORECASE ):
#         val = "Handbag"
#     elif re.search(r'scar(?:f|ves)|bandana', txt, re.IGNORECASE ):
#         val = "Scarf"
#     return val
# product_copy["product_category"] = product_copy.description.apply( findCategory )
# product_copy["product_category"].value_counts()

In [39]:
## Joyi's category creation code

bottom_seq=r'\b(capri?|leggings?|bottoms?|skirts?|sweatpants?|pants?|jeans?|midi|trousers?|shorts?|trunks?)\b' #|knee|ankle
one_piece_seq=r'\b(kimono|jumpsuit|dress(?:es)?|gowns?|swimsuit?|onesies?|unitards?|bodysuits?|rompers?|one ?piece)\b'
shoe_seq=r'\b(shoes?|sneakers?|flats|boot|heels?|sandals?|mules?|loafers?|pumps?)\b' #(?:ies|s)?
handbag_seq=r'\b(bags?|handbags?|clutch(?:es)?|wallets?|purses?|duffels?)\b'
scarf=r'\b(scar(?:f|ves)?|wraps?|stoles?|shawl)\b' # row 412, 705, 
top=r'\b(Tank|caftan|hoodies?|tshirts?|tees?|tops?|top|blouses?|jackets?|blazers?|shirts?|tops?|coats?|suits|sweaters?|sweatshirts?)\b'
acc=r'\b(capes?|socks?|earrings?|belts?|gloves?|headbands?|ties?|hats?|caps?)\b'
linen=r'\b(linens?)\b'
lingerie=r'\b(bras?)\b'
# product_copy['exclusion']=product_copy['name'].str.contains(list_stop,case=False)


# product category hierarchy
dict_seq={'one_piece':one_piece_seq,
          'shoe':shoe_seq,
          'handbag':handbag_seq,
          'scarf':scarf,
          'top':top,'acc':acc,'linen':linen,
          'bottom':bottom_seq,'lingerie':lingerie
        }


# occurence score calculation
for d in dict_seq:
    product_copy[f'{d}_check']=0
#     print(f'{d}_check')
    for col in ['name','description','details','brand_category']:
        product_copy[f'{d}_check']=product_copy[f'{d}_check']+product_copy[col].str.contains(dict_seq[d],case=False)
#         print(product_copy[col].str.contains(dict_seq[d],case=False))   


 #calculating max occurence         
product_copy['max_value_cat']=product_copy[["bottom_check","shoe_check","one_piece_check","handbag_check","scarf_check",'acc_check','top_check','linen_check','lingerie_check']].max(axis=1)
def max_presence(row):
    for d in dict_seq.keys():
        colname=f'{d}_check'
        if (row['max_value_cat']==row[colname])&(row['max_value_cat']!=0):
            return d
        elif row['max_value_cat']==0:
            return 'None'
#searching for the category with max occurence    
product_copy['final_category']=product_copy.apply(max_presence,axis=1)

C:\Users\Gordon\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [40]:
product_copy['final_category'].value_counts()

top          18350
None         12882
one_piece     9694
bottom        8667
shoe          4899
acc           2762
handbag       2500
scarf          857
lingerie       435
linen          307
Name: final_category, dtype: int64

## Color

In [41]:
def findColors(txt):
    """ Function to determine the color of item """
    #colors_re = r'\bBeige\b|\bBlack\b|\bBlue\b|\bBrown\b|\bBurgundy\b|\bGold\b|\bGray\b|\bGreen\b|\bMulticolor|\bNavy\b|\bNeutral\b|\bOrange\b|\bPinks\b|\bPurple\b|\bRed\b|\bSilver\b|\bTeal\b|\bWhite\b|\bYellow\b'
    colors_re = r'\b(beige|light brown|black|blue ?green|blue|brown|umber|burgundy|gold(?:en)?|gray|grey|green|navy|neutral|orange|aurantia|pink|purple|violet|red|scarlet|silver|teal|white|yellow|(?:multi(?:ple)?|several|different|many|more than one) ?colou?rs?)\b'

    val = []
    txt = str(txt)
    if re.findall(colors_re, txt, re.IGNORECASE ):
        val = re.findall(colors_re, txt, re.IGNORECASE )
    return val

In [42]:
# find all colors in item descriptions
product_copy['color_list'] = product_copy['description'].apply(findColors) +product_copy['name'].apply(findColors)              
# standardize all words to lowercase and make the list a set to avoid repeated colors

# number of unique colors in the set
# product_copy['n_colors'] = product_copy['color_list'].apply(len)
# product_copy[['color_list','n_colors']].head(10)

In [43]:
product_copy['colors'] = product_copy['color_list'].apply(lambda x: set(y.lower() for y in x))
#product_copy.colors = product_copy['color_list'].replace(set(), np.nan)

In [44]:
product_copy.sample(50)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,...,scarf_check,top_check,acc_check,linen_check,bottom_check,lingerie_check,max_value_cat,final_category,color_list,colors
16010,01EC8M5B6ZYFXBH3CPGZ90A99P,Studio 189,Unknown,Black Xan Silk Men's Style Andy Pants,None,2020-07-02 20:40:49.113 UTC,https://fashion-rising-collection.myshopify.co...,product details 100 cotton grown in ghana slim...,Product Details \n\n100% Cotton grown in Ghan...,Black Xan Silk Men's Style Andy Pants,...,0,0,1,0,2,0,2.0,bottom,"[black, black, Black]",{black}
42343,01EQ9PBBJW4CFH9BTJTQ1VDSTN,7 For All Mankind,Unknown,MITERED POCKET TEE,None,2020-11-16 23:30:57.756 UTC,https://www.example.com/1054285328,none,None,MITERED POCKET TEE,...,0,1,0,0,0,0,1.0,top,[],{}
15619,01EYXWGEZBZ62GSJ3D397WRW0M,M.M.LaFleur,Jewelry,Ponza Earrings - Silver,None,2021-02-19 19:33:17.931 UTC,https://mmlafleur.com/shop/catalog/product/vie...,think studs modern take classic pearl earrings...,Think of these studs as a modern take on your ...,Ponza Earrings - Silver,...,0,0,2,0,0,0,2.0,acc,"[silver, Silver]",{silver}
57140,01EY1JCSYP5FCYBRAMAREVMQMX,Rachel Comey,Unknown,Danziger Skirt,None,2021-02-08 19:37:48.246 UTC,https://rachel-comey.myshopify.com/products/da...,danziger skirt pleat front trouser skirt in fw...,The Danziger Skirt is a pleat front trouser sk...,Danziger Skirt,...,0,0,0,0,2,0,2.0,bottom,[],{}
54068,01EC0GSZ012RHKRM9TAYBNE60T,BROCHU WALKER,Unknown,The Herve Skirt,None,2020-06-29 17:08:14.969 UTC,https://brochuwalker.myshopify.com/products/he...,effortless graceful style in bold new hue herv...,"An effortless and graceful style, in a bold ne...",The Herve Skirt,...,0,0,1,0,2,0,2.0,bottom,[],{}
15112,01E4ECWQCPRJGCJ6GRWV2HF5QS,Rails,TOP,Rebel,"DETAILS |\nLong sleeve, 100% silk pajama style...",2020-03-27 16:23:38.638 UTC,https://www.railsclothing.com/products/rebel-r...,none,None,Rebel,...,0,2,0,0,0,0,2.0,top,[],{}
44568,01ED5A7JK8WBF6HDY3QCWSDWMZ,lemlem,Unknown,Selina Split Wrap,None,2020-07-14 00:05:14.976 UTC,https://lemlem.myshopify.com/products/selina-s...,style details multi purpose travel garment mad...,STYLE DETAILS\n\n • A multi-purpose travel gar...,Selina Split Wrap,...,1,0,0,0,0,0,1.0,scarf,[],{}
27384,01EYV5BX1M725K03ZRVR428YK1,Christina Lehr,Unknown,Kiernan Turtle,None,2021-02-18 18:10:22.388 UTC,https://christina-lehr.myshopify.com/products/...,birch sweater refined birch sweater knitted fi...,Birch Sweater \n\nThe refined Birch Sweater is...,Kiernan Turtle,...,0,1,0,0,1,0,1.0,top,[neutral],{neutral}
178,01ESVM02ZC6T7GJR6GN796Q9GY,Warm,Unknown,mens surf yourself clean tee,None,2020-12-18 19:07:34.764 UTC,https://shopwarm.myshopify.com/products/mens-s...,tie dye tee horizontal multi color dye vertica...,tie dye tee. horizontal multi-color dye with a...,mens surf yourself clean tee,...,0,2,1,0,0,0,2.0,top,"[multi color, green, brown]","{green, multi color, brown}"
17870,01EG6RDV70WTBQQTK9MAY4JDDC,Tanya Taylor,Unknown,ADONICA TOP - FAN FLORAL NAVY MULTI,None,2020-08-20 20:19:40.377 UTC,https://tanyataylor2017.myshopify.com/products...,adonica top,ADONICA TOP,ADONICA TOP - FAN FLORAL NAVY MULTI,...,0,2,0,0,0,0,2.0,top,[NAVY],{navy}


In [45]:
product_copy['n_colors'] = product_copy['colors'].apply(len)
product_copy[['colors','n_colors']].head(10)



,colors,n_colors
0,"{black, white}",2
1,{pink},1
2,{red},1
3,{black},1
4,{black},1
5,{},0
6,{},0
7,{},0
8,{},0
9,{},0


In [46]:


# tag color labels
product_copy['final_color'] = np.nan

# label items with more than one color as "Multi"
product_copy.loc[product_copy['n_colors']>1,'final_color'] = 'multi'

# label items with one color
product_copy.loc[product_copy['n_colors']==1,'final_color'] = product_copy.loc[product_copy['n_colors']==1,'colors'].\
                                                                        apply(lambda x:list(x)[0])

In [47]:
product_copy['final_color'].value_counts()

multi                5416
black                4301
white                1990
blue                 1982
gold                  938
navy                  761
green                 759
pink                  734
grey                  604
red                   542
silver                401
yellow                386
brown                 309
orange                267
neutral               225
beige                 149
golden                139
purple                129
multicolor            113
gray                  108
violet                102
burgundy               89
multi color            64
teal                   56
blue green             10
different colors        9
scarlet                 8
different colours       7
multicolour             6
light brown             6
multiple colors         5
different color         4
several colours         2
multiple color          1
many colors             1
umber                   1
many colours            1
different colour        1
multi colors

In [48]:
multi_pattern = r'\b(?:multi(?:ple)?|several|different|many|more than one\b)\s? ?colou?rs?'
for i in tqdm(range(len(product_copy))):
    match = re.search(multi_pattern,
                       str(product_copy.loc[i,'final_color']))
    if match:
        product_copy.loc[i,'final_color'] = 'multi'
                       
    

100%|██████████| 61355/61355 [00:00<00:00, 75716.16it/s]


In [49]:
product_copy['final_color'].value_counts()

multi          5631
black          4301
white          1990
blue           1982
gold            938
navy            761
green           759
pink            734
grey            604
red             542
silver          401
yellow          386
brown           309
orange          267
neutral         225
beige           149
golden          139
purple          129
gray            108
violet          102
burgundy         89
teal             56
blue green       10
scarlet           8
light brown       6
umber             1
Name: final_color, dtype: int64

In [50]:
# # define a function to standardize color labels
# def color_tag(text):
    
#     color = text
    
#     if isinstance(text, str):

#         if text == 'beige' or text == 'light brown':
#             color = 'beige'
#         elif text == 'blue green' or text == 'bluegreen' or text == 'teal':
#             color = 'teal'
#         elif text == 'brown' or text == 'umber':
#             color = 'brown'
#         elif text == 'gold' or text == 'golden':
#             color = 'gold'
#         elif text == 'gray' or text == 'grey':
#             color = 'gray'
#         elif text == 'orange' or text == 'aurantia':
#             color = 'orange'
#         elif text == 'purple' or text == 'violet':
#             color = 'purple'
#         elif text == 'red' or text == 'scarlet':
#             color = 'red'
#         elif re.findall(r"(?:multi(?:ple)?|several|different|many|more than one) ?colou?rs?\b", text) != []:
#             color = 'Multi'
#         else:
#             color = text.capitalize()
    
#     return color

# product_copy['colors'] = product_copy['colors'].apply(color_tag)
# product_copy[['color_list', 'n_colors', 'colors']].head(10)

## dry clean, hand wash, tumble dry...

In [51]:
# Machine wash cold, tumble dry low,Hand Wash Cold. Hang Dry. No Bleach

In [52]:
def findCare(txt):
    """ Function to determine the care method of item """
    txt = str(txt)
    val = []
    care_re = r'\b(dry clean|hand wash|not? bleach|tumble dry|hang dry\
                |machine wash cold|machine wash|(?:[don\'t|not?]) machine wash)\b'
    if re.findall(care_re, txt, re.IGNORECASE ):
        val = re.findall(care_re, txt, re.IGNORECASE)
    return val

In [53]:
product_copy['care_list'] = product_copy['description'].apply(findCare)              
product_copy['cares'] = product_copy['care_list'].apply(lambda x: set(y.lower() for y in x))


In [54]:
product_copy['cares'].value_counts()

TypeError: unhashable type: 'set'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'set'


{}                                                       48122
{dry clean}                                               6094
{hand wash}                                               2313
{tumble dry, machine wash cold}                           1696
{hand wash, dry clean}                                    1123
{machine wash cold}                                        582
{tumble dry, dry clean, machine wash cold}                 466
{hand wash, machine wash}                                  317
{tumble dry}                                               283
{machine wash}                                             170
{dry clean, machine wash cold}                              79
{tumble dry, machine wash}                                  37
{tumble dry, hand wash}                                     16
{tumble dry, hand wash, dry clean}                          13
{hand wash, dry clean, machine wash}                        12
{dry clean, machine wash}                              

the care methods are: 
hand wash,dry clean,machine wash cold,tumble dry


In [55]:
def is_dry_clean(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(dry clean)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_dry_clean'] = pd.DataFrame(product_copy.description.apply(is_dry_clean))
product_copy['is_dry_clean'].value_counts()

False    53542
True      7813
Name: is_dry_clean, dtype: int64

In [56]:
def is_hand_wash(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(handwash)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_hand_wash'] = pd.DataFrame(product_copy.description.apply(is_hand_wash))
product_copy['is_hand_wash'].value_counts()

False    61282
True        73
Name: is_hand_wash, dtype: int64

In [57]:
def is_machine_wash_cold(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(is_machine_wash_cold)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_machine_wash_cold'] = pd.DataFrame(product_copy.description.apply(is_machine_wash_cold))
product_copy['is_machine_wash_cold'].value_counts()

False    61355
Name: is_machine_wash_cold, dtype: int64

In [58]:
def is_machine_wash_cold(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(is_machine_wash_cold)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_machine_wash_cold'] = pd.DataFrame(product_copy.description.apply(is_machine_wash_cold))
product_copy['is_machine_wash_cold'].value_counts()

False    61355
Name: is_machine_wash_cold, dtype: int64

In [59]:
def is_machine_wash(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(is machine wash)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_machine_wash'] = pd.DataFrame(product_copy.description.apply(is_machine_wash))
product_copy['is_machine_wash'].value_counts()

False    61355
Name: is_machine_wash, dtype: int64

In [60]:
def is_tumble_dry(txt):
    txt = str(txt)
    val = 'False'
    if re.search(r'\b(tumble dry)\b', txt, re.IGNORECASE ):
        val = 'True'
   
    return val

product_copy['is_tumble_dry'] = pd.DataFrame(product_copy.description.apply(is_tumble_dry))
product_copy['is_tumble_dry'].value_counts()

False    58828
True      2527
Name: is_tumble_dry, dtype: int64

In [61]:
product_copy.columns

Index(['product_id', 'brand', 'brand_category', 'name', 'details',
       'created_at', 'brand_canonical_url', 'description', 'brand_description',
       'brand_name', 'product_active', 'is_womens_clothing',
       'is_children_clothing', 'is_men_clothing', 'user_type',
       'one_piece_check', 'shoe_check', 'handbag_check', 'scarf_check',
       'top_check', 'acc_check', 'linen_check', 'bottom_check',
       'lingerie_check', 'max_value_cat', 'final_category', 'color_list',
       'colors', 'n_colors', 'final_color', 'care_list', 'cares',
       'is_dry_clean', 'is_hand_wash', 'is_machine_wash_cold',
       'is_machine_wash', 'is_tumble_dry'],
      dtype='object')

## made in, Is USA

In [62]:
def made_in(txt):
    txt = str(txt)
    loc = re.findall(r'(?:\bmade\b\s\bin\b\s)(?:the\s)?(\b\w+\b)', txt, re.IGNORECASE )
    return loc

#loc = re.findall(r'\bmade\sin(?:\s\b(\w+)\b,)?\s(\b\w+\b)', txt, re.IGNORECASE )

In [63]:
made_in(product_copy.description[1])

['new_york_city']

In [64]:
product_copy['made_in'] = pd.DataFrame(product_copy.description.apply(made_in))
product_copy['made_in'].value_counts()[:40]


TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                 45556
[USA]               5495
[italy]             1473
[china]              914
[peru]               769
[ethiopia]           737
[la]                 648
[los]                634
[nyc]                613
[spain]              430
[new_york_city]      375
[portugal]           364
[india]              331
[kenya]              331
[accra]              314
[brazil]             197
[u]                  162
[turkey]             133
[morocco]            125
[america]            122
[italy, italy]       110
[vietnam]            107
[france]             105
[accra, mali]         54
[madagascar]          48
[new]                 43
[argentina]           42
[brooklyn, nyc]       41
[prc]                 38
[japan]               35
[partnership]         30
[soft]                29
[california]          29
[sizing]              28
[pt]                  27
[spain, spain]        25
[los, la]             24
[bali]                24
[lima]                23
[factory]             21


In [65]:
#question: if we create a feature 'ismadeinUSA', and we find all the products made in
# la, nyc, label them as 'true' in 'ismadeinUSA'
# will it be helpful if the future descriptions using city names rather than USA?

In [66]:

product_copy['n_made_in'] = product_copy['made_in'].apply(len)
product_copy['final_made_in'] = np.nan
product_copy.loc[product_copy['n_made_in']>=1,'final_made_in'] = product_copy.loc[product_copy['n_made_in']>=1,'made_in'].\
                                                                        apply(lambda x:list(x)[0])

In [67]:
product_copy['final_made_in'].value_counts()

USA           5507
italy         1585
china          914
peru           789
ethiopia       737
              ... 
recover          1
ethiopioa        1
tredate          1
completely       1
sharkskin        1
Name: final_made_in, Length: 176, dtype: int64

In [68]:
def is_USA(txt):
    val = []
    txt = str(txt)
    #print(txt)
    if txt in ['USA','la','nyc','new_york_city','brooklyn','los','california']:
        val = 'True'
    else:
        val = 'False'
    
    return val
    
# product_copy['is_made_usa'] = np.where(product_copy.final_made_in in ['USA','la','nyc','new_york_city','brooklyn','los','california'],
#         'True','False')
product_copy['is_made_usa'] = pd.DataFrame(product_copy.final_made_in.apply(is_USA))
product_copy['is_made_usa'].value_counts()       

False    53465
True      7890
Name: is_made_usa, dtype: int64

## fabric: 'poly chiffon','linen','cashmere' 

In [69]:
fabric_list = ['broadcloth','brocade','calico','cashmere','chambray','chiffon',
               'corduroy','cotton','eyelet','faille','foulard','furbelow',
               'fustian','gingham','grosgrain','jacquard','knit','linen','lisle',
              'madras','merino','paisley','sateen','satin','seersucker','shetland',
              'silk','taffeta','tulle','velvet','polyester']

In [70]:

def findFabric(txt):
    """ Function to determine the fabric of item """
    fabric_re = r'\b(broadcloth|brocade|calico|cashmere|chambray|chiffon|corduroy|cotton|eyelet|faille|foulard|furbelow|fustian|gingham|grosgrain|jacquard|knit\
                    |linen|lisle|madras|merino|paisley|sateen|satin|\
                    seersucker|shetland|silk|taffeta|tulle|velvet|polyester|rayon)\b'
    val = []
    txt = str(txt)
    if re.findall(fabric_re, txt, re.IGNORECASE ):
        val = re.findall(fabric_re, txt, re.IGNORECASE )
    return val

In [71]:
product_copy['fabric_list'] = product_copy['description'].apply(findFabric) +product_copy['name'].apply(findFabric)              

product_copy['fabrics'] = product_copy['fabric_list'].apply(lambda x: set(y.lower() for y in x))


In [72]:
product_copy['fabrics'].value_counts()


TypeError: unhashable type: 'set'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'set'


{}                             36472
{cotton}                       10781
{polyester}                     1998
{silk}                          1663
{cashmere}                       894
                               ...  
{cashmere, silk, polyester}        1
{merino, shetland}                 1
{merino, rayon}                    1
{gingham, linen}                   1
{linen, paisley}                   1
Name: fabrics, Length: 248, dtype: int64

In [73]:
product_copy['n_fabric'] = product_copy['fabrics'].apply(len)

In [74]:

# tag color labels
product_copy['final_fabric'] = np.nan
# label items with more than one color as "Multi"
product_copy.loc[product_copy['n_fabric']>1,'final_fabric'] = 'multi'
# label items with one color
product_copy.loc[product_copy['n_fabric']==1,'final_fabric'] = \
       product_copy.loc[product_copy['n_fabric']==1,'fabrics'].apply(lambda x:list(x)[0])
  

In [75]:
product_copy['final_fabric'].value_counts()

cotton       10781
multi         5358
polyester     1998
silk          1663
cashmere       894
linen          886
chiffon        739
rayon          647
satin          374
merino         277
velvet         269
chambray       251
jacquard       206
grosgrain      132
corduroy        99
eyelet          95
gingham         67
sateen          47
paisley         30
tulle           29
taffeta         22
foulard          9
brocade          3
madras           3
calico           2
lisle            1
faille           1
Name: final_fabric, dtype: int64

## style

In [76]:
product_copy.columns

Index(['product_id', 'brand', 'brand_category', 'name', 'details',
       'created_at', 'brand_canonical_url', 'description', 'brand_description',
       'brand_name', 'product_active', 'is_womens_clothing',
       'is_children_clothing', 'is_men_clothing', 'user_type',
       'one_piece_check', 'shoe_check', 'handbag_check', 'scarf_check',
       'top_check', 'acc_check', 'linen_check', 'bottom_check',
       'lingerie_check', 'max_value_cat', 'final_category', 'color_list',
       'colors', 'n_colors', 'final_color', 'care_list', 'cares',
       'is_dry_clean', 'is_hand_wash', 'is_machine_wash_cold',
       'is_machine_wash', 'is_tumble_dry', 'made_in', 'n_made_in',
       'final_made_in', 'is_made_usa', 'fabric_list', 'fabrics', 'n_fabric',
       'final_fabric'],
      dtype='object')

# Lemma & Stemming
4.1,4.2 are parallel  

In [77]:
import nltk
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

## Stemming

In [78]:
stemmer = PorterStemmer()

In [79]:
def stemming_sentence(sentence):
    sentence = str(sentence)
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    stemming_sentence = []
    for word, tag in nltk_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            stemming_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            stemming_sentence.append(stemmer.stem(word))
    return stemming_sentence

In [80]:
# porter=PorterStemmer()

# def stemSentence(sentence):
#     sentence = str(sentence)
#     token_words=word_tokenize(sentence)
#     token_words
#     stem_sentence=[]
#     for word in token_words:
#         stem_sentence.append(porter.stem(word))
#         stem_sentence.append(" ")
#     return "".join(stem_sentence)

# print(my_lines_list[0])
# print("Stemmed sentence")
# x=stemSentence(my_lines_list[0])
# print(x)

In [81]:
%%time
stemmed_descriptions  = []
for des in product_copy.description:
    stemmed_description = stemming_sentence(des)
    stemmed_descriptions.append(stemmed_description)

Wall time: 3min 5s


In [82]:
#stemmed_descriptions

In [83]:
stemmed_descriptions_join = []
for doc in tqdm(stemmed_descriptions):
    new_doc = ' '.join(doc)
    stemmed_descriptions_join.append(new_doc)

100%|██████████| 61355/61355 [00:00<00:00, 735307.71it/s]


In [84]:
product_copy['stemmed_description'] = stemmed_descriptions_join 

In [85]:
product_copy.head(2)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,...,is_tumble_dry,made_in,n_made_in,final_made_in,is_made_usa,fabric_list,fabrics,n_fabric,final_fabric,stemmed_description
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,None,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,signature khadi shirt available in black white...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,...,False,[],0,NaN,False,"[cotton, cotton]",{cotton},1,cotton,signatur khadi shirt avail in black white easi...
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,None,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,mid length dress ruffles adjustable straps bia...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,...,False,[new_york_city],1,new_york_city,True,[],{},0,NaN,mid length dress ruffl adjust strap bia cut si...


## Lemmatization with wordnet tag

In [86]:

# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
def lemmatize_sentence(sentence):
    sentence = str(sentence)
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [87]:
## don't use list comprehension, save memory 
lemmatized_descriptions = []
for des in product_copy.description:
    lemmatized_description =  lemmatize_sentence(des)
    lemmatized_descriptions.append(lemmatized_description)
    

#lemmatized_descriptions = [lemmatize_sentence(des) for des in product_copy.description]

In [88]:
## the lemmatized_description is a list of list with all the words separated 
## so we need to join them together as a complete sentence
lemmatized_description_join = []
for doc in tqdm(lemmatized_descriptions):
    new_doc = ' '.join(doc)
    lemmatized_description_join.append(new_doc)

100%|██████████| 61355/61355 [00:00<00:00, 706871.51it/s]


In [89]:
lemmatized_description_join

['signature khadi shirt available in black white easy beach city promise top go warm weather item perfect blazer hand loom woven stripe in khadi cotton slightly sheer get soft every wash ship first week april color white black length 27 width 26 5 one fit grid khadi cotton',
 'mid length dress ruffle adjustable strap bias cut side seam invisible zipper make in new_york_city wear 100 rise sylk rise sylk organic cellulose fiber make natural waste rise bush stem',
 'ibi slip raw red knit sneaker woman',
 'ibi slip black knit sneaker woman',
 'none',
 'easy smock mini dress gorgeous puff sleeves feel like first day spring belt include 100 linen wear x',
 'picture perfect clara stun linen cover call sandy foot spicy cocktail disappoint 100 linen wear style dual sizing pick x l',
 'best sell bump friendly simone favourite in time difficult anything else say warn 100 cotton wear x',
 'super appropriate length bit cold shoulder although stretch back dress snug fit bra need yes please consider 

In [90]:
product_copy['lemmatized_description'] = lemmatized_description_join

In [91]:
product_copy.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,...,made_in,n_made_in,final_made_in,is_made_usa,fabric_list,fabrics,n_fabric,final_fabric,stemmed_description,lemmatized_description
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,None,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,signature khadi shirt available in black white...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,...,[],0,NaN,False,"[cotton, cotton]",{cotton},1,cotton,signatur khadi shirt avail in black white easi...,signature khadi shirt available in black white...
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,None,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,mid length dress ruffles adjustable straps bia...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,...,[new_york_city],1,new_york_city,True,[],{},0,NaN,mid length dress ruffl adjust strap bia cut si...,mid length dress ruffle adjustable strap bias ...
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,None,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,ibi slip raw red knit sneaker women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,...,[],0,NaN,False,[],{},0,NaN,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker woman
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,None,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,ibi slip black knit sneaker women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,...,[],0,NaN,False,[],{},0,NaN,ibi slip black knit sneaker women,ibi slip black knit sneaker woman
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,None,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,none,None,CATIBA PRO Skate Black Suede and Canvas Contra...,...,[],0,NaN,False,[],{},0,NaN,none,none


# Gordon's Features

In [130]:
import pandas as pd 
import regex as re


Created a feature that includes the city/country the item is made in. Countries such as Italy, Ghana, India, and Spain were included because we saw a few items that had associations with those countries.

In [175]:
location_list_clean = []
for rows in range(0,len(product)):
    #check if content in description column, if not (aka null), there's usually content in details column
    if pd.isnull(product.loc[rows,'description']):
        if pd.isnull(product.loc[rows,'details']):
            location_list_clean.append('None')
            continue
        else:
            #append entity to list and operate on the details column, will be used to slap it onto main dataframe later
            if len(re.findall(r'\b(USA|United States)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('USA')
            elif len(re.findall(r'\b(Italy)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Italy')
            elif len(re.findall(r'\b(New York|NewYork|NewYorkCity|NY|NYC|N\.Y\.)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('New York')
            elif len(re.findall(r'\b(Los Angeles|LA|LosAngeles|L\.A\.)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('LA')
            elif len(re.findall(r'\b(Ghana)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Ghana')
            elif len(re.findall(r'\b(China)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('China')
            elif len(re.findall(r'\b(India)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('India')
            elif len(re.findall(r'\b(Spain)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Spain')
            elif len(re.findall(r'\b(France)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('France')
            elif len(re.findall(r'\b(London|UK|U\.K\.)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('London/UK')
            elif len(re.findall(r'\b(Japan|Tokyo)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Japan')
            else:
                location_list_clean.append('None')
    else:
        #do the same thing as above if content is in description column
            if len(re.findall(r'\b(USA|United States)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('USA')
            elif len(re.findall(r'\b(Italy)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Italy')
            elif len(re.findall(r'\b(New York|NewYork|NewYorkCity|NY|NYC|N\.Y\.)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('New York')
            elif len(re.findall(r'\b(Los Angeles|LA|LosAngeles|L\.A\.)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('LA')
            elif len(re.findall(r'\b(Ghana)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Ghana')
            elif len(re.findall(r'\b(China)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('China')
            elif len(re.findall(r'\b(India)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('India')
            elif len(re.findall(r'\b(Spain)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Spain')
            elif len(re.findall(r'\b(France)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('France')
            elif len(re.findall(r'\b(London|UK|U\.K\.)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('London/UK')
            elif len(re.findall(r'\b(Japan|Tokyo)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                location_list_clean.append('Japan')
            else:
                location_list_clean.append('None')

Occasions could be an interesting feature, since brands might have specialties and/or some brands are very occasion specific (ex. yoga attire only, swimwear only, etc). We wanted to capture all four seasons, while further segmenting Work, Beach, Swimwear, Sports, and Yoga.

In [176]:
occasion_list_cleaned = []
for rows in range(0,len(product)):
    #check if content in description column, if not (aka null), there's usually content in details column
    if pd.isnull(product.loc[rows,'description']):
        if pd.isnull(product.loc[rows,'details']):
            occasion_list_cleaned.append('None')
            continue
        else:
            #append entity to list and operate on the details column, will be used to slap it onto main dataframe later
            if len(re.findall(r'\b(summer|summertime|sun|sunny|heat|June|July|August)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Summer_Season')
            elif len(re.findall(r'\b(work)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Work')
            elif len(re.findall(r'\b(beach|beachy|sand|sandy)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Beach')
            elif len(re.findall(r'\b(fall|autumn|thanksgiving|halloween|September|October|November)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Fall_Season')
            elif len(re.findall(r'\b(spring|March|April|Easter)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Spring_Season')
            elif len(re.findall(r'\b(winter|cold|New Year|Christmas|snow|December|January|February)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Winter_Season')                
            elif len(re.findall(r'\b(swim|water|wet|pool|swimwear)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Swimwear')    
            elif len(re.findall(r'\b(sports|golf|tennis|marathon|basketball|soccer|cycling|hiking|climbing|running|sport|sportswear|hike|climb|run|workout)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Sports') 
            elif len(re.findall(r'\b(yoga|studio|yogi|poses)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Yoga')
            else:
                occasion_list_cleaned.append('None')
    else:
        #do the same thing as above if content is in description column
            if len(re.findall(r'\b(summer|summertime|sun|sunny|heat|June|July|August)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Summer_Season')
            elif len(re.findall(r'\b(work)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Work')
            elif len(re.findall(r'\b(beach|beachy|sand|sandy)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Beach')
            elif len(re.findall(r'\b(fall|autumn|thanksgiving|halloween|September|October|November)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Fall_Season')
            elif len(re.findall(r'\b(spring|March|April|Easter)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Spring_Season')
            elif len(re.findall(r'\b(winter|cold|New Year|Christmas|snow|December|January|February)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Winter_Season')                
            elif len(re.findall(r'\b(swim|water|wet|pool|swimwear)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Swimwear')    
            elif len(re.findall(r'\b(sports|golf|tennis|marathon|basketball|soccer|cycling|hiking|climbing|running|sport|sportswear|hike|climb|run|workout)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Sports') 
            elif len(re.findall(r'\b(yoga|studio|yogi|poses)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >= 1:
                occasion_list_cleaned.append('Yoga')
            else:
                occasion_list_cleaned.append('None')

We noticed some brands used deadstock fabrics/materials to create items, so we wanted to see if this could help predict the brand better.

In [177]:
deadstock_list = []
for rows in range(0,len(product)):
    #check if content in description column, if not (aka null), there's usually content in details column
    if pd.isnull(product.loc[rows,'description']):
        if pd.isnull(product.loc[rows,'details']):
            deadstock_list.append(0)
            continue
        else:
            #append entity to list and operate on the details column, will be used to slap it onto main dataframe later
            if len(re.findall(r'\b(deadstock)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >=1:
                deadstock_list.append(1)
            else:
                deadstock_list.append(0)

    else:
        #do the same thing as above if content is in description column
            if len(re.findall(r'\b(deadstock)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >=1:
                deadstock_list.append(1)
            else:
                deadstock_list.append(0)

Similar to the logic of trying the deadstock feature, we noticed some brands handcrafted their items, so we wanted to see if this could help predict the brand better.

In [178]:

handcrafted_list = []
for rows in range(0,len(product_copy)):
    #check if content in description column, if not (aka null), there's usually content in details column
    if pd.isnull(product.loc[rows,'description']):
        if pd.isnull(product.loc[rows,'details']):
            handcrafted_list.append(0)
            continue
        else:
            #append entity to list and operate on the details column, will be used to slap it onto main dataframe later
            if len(re.findall(r'\b(handcrafted|hand-crafted)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >=1:
                handcrafted_list.append(1)
            else:
                handcrafted_list.append(0)

    else:
        #do the same thing as above if content is in description column
            if len(re.findall(r'\b(handcrafted|hand-crafted)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >=1:
                handcrafted_list.append(1)
            else:
                handcrafted_list.append(0)

Some brands had many items that fit true to size, so we wanted to create a feature from this as well.

In [179]:
truetosize_list = []
for rows in range(0,len(product)):
    #check if content in description column, if not (aka null), there's usually content in details column
    if pd.isnull(product.loc[rows,'description']):
        if pd.isnull(product.loc[rows,'details']):
            truetosize_list.append(0)
            continue
        else:
            #append entity to list and operate on the details column, will be used to slap it onto main dataframe later
            if len(re.findall(r'\b(true to size)\b',str(product.loc[rows,'details']),flags=re.IGNORECASE)) >=1:
                truetosize_list.append(1)
            else:
                truetosize_list.append(0)

    else:
        #do the same thing as above if content is in description column
            if len(re.findall(r'\b(true to size)\b',str(product.loc[rows,'description']),flags=re.IGNORECASE)) >=1:
                truetosize_list.append(1)
            else:
                truetosize_list.append(0)

In [180]:
product_copy['Made_In'] = location_list_clean
product_copy['Occasion'] = occasion_list_cleaned
product_copy['Deadstock'] = deadstock_list
product_copy['Handcrafted'] = handcrafted_list
product_copy['True_To_Size'] = truetosize_list

# Brand Encoding-- the top brands

In [99]:
brand_analysis=pd.DataFrame(cleaned['brand'].astype(str).value_counts())
Brand_to_classify=list(brand_analysis.head(30).index)

NameError: name 'cleaned' is not defined

In [ ]:
def brand_exclusion(row):
    if row in Brand_to_classify:
        return row
    else:
        return 'Other'
cleaned['label']=cleaned['brand'].apply(brand_exclusion)
cleaned['label'].astype(str).value_counts()

In [182]:
product_copy.to_csv('product_with_feature_Stem_Lemma_Sue_version3.csv')